In [ ]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ▶ Cell X: Build nearest-neighbor 6-h features for 2023
import pandas as pd
from pathlib import Path

# 1) paths – adjust to your Drive
SRC = Path("/content/drive/MyDrive/preof2023.csv")
DST = Path("/content/drive/MyDrive/meteo_2023_last6h.csv")

# 2) variables (in your CSV)
vars = [
    "air_temp_C",
    "dew_point_C",
    "relative_humidity_percent",
    "wind_speed_m_s",
    "precipitation_in"
]

# 3) load & prepare
df = pd.read_csv(SRC, parse_dates=['datetime'])
df['date'] = df['datetime'].dt.date

rows = []
for day, grp in df.groupby('date'):
    grp = grp.sort_values('datetime')
    row = {'date': day}

    # for each lag = 5 → 0
    for lag in range(5, -1, -1):
        target = pd.Timestamp(day) + pd.Timedelta(hours=10 - lag)

        # find nearest timestamp in grp
        diffs = (grp['datetime'] - target).abs()
        idx   = diffs.idxmin()
        rec   = grp.loc[idx]

        # record each variable
        for v in vars:
            row[f"{v}_t-{lag}h"] = rec[v]

    rows.append(row)

# 4) assemble & save
feat_df = pd.DataFrame(rows)
feat_df.to_csv(DST, index=False)
print(f"✅ wrote {len(feat_df)} rows → {DST}")


✅ wrote 365 rows → /content/drive/MyDrive/meteo_2023_last6h.csv


In [ ]:
# Cell 1 ▶ Imports & Test DataLoader
import os, math
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tifffile import imread

# ▶ point to your test CSV + patches folder
TEST_CSV     = "/content/drive/MyDrive/meteo_2023_last6h.csv"
PATCHES_DIR  = "/content/drive/MyDrive/test_patches"

# ▶ read & pick only the 6-h lag cols
df_test = pd.read_csv(TEST_CSV, parse_dates=['date'])
seq_cols = [c for c in df_test.columns if "_t-" in c]

class LSTTestDataset(Dataset):
    def __init__(self, df, patches_dir, seq_cols, n_hours=6):
        self.df          = df.reset_index(drop=True)
        self.patches_dir = patches_dir
        self.seq_cols    = seq_cols
        self.n_hours     = n_hours
        self.n_vars      = len(seq_cols)//n_hours
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        row = self.df.loc[i]
        arr = imread(os.path.join(self.patches_dir, row["patch_filename"])).astype(np.float32)
        img = torch.from_numpy(arr[[1,2,3]]).float()    # [3,H,W]
        tar = torch.from_numpy(arr[0]).unsqueeze(0).float()  # [1,H,W]
        w_flat = row[self.seq_cols].values.astype(np.float32)
        w_seq  = torch.from_numpy(w_flat).view(self.n_hours, self.n_vars)
        return img, w_seq, tar

test_ds = LSTTestDataset(df_test, PATCHES_DIR, seq_cols)
test_loader = DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=0, pin_memory=False)
print("Test samples:", len(test_ds), "→ Batches:", len(test_loader))

Test samples: 365 → Batches: 92


In [ ]:
# Cell 2 ▶ Model definition (ViT+GRU variant)
import torch.nn as nn
import timm

class PretrainedViTLSTModel(nn.Module):
    def __init__(self, weather_dim=5, hidden_dim=768,
                 vit_name="vit_base_patch16_224",
                 num_layers=2, num_heads=8):
        super().__init__()
        self.vit = timm.create_model(vit_name, pretrained=False, num_classes=0)
        self.weather_encoder = nn.GRU(input_size=weather_dim,
                                      hidden_size=hidden_dim,
                                      batch_first=True)
        enc = nn.TransformerEncoderLayer(d_model=hidden_dim,
                                         nhead=num_heads,
                                         dim_feedforward=hidden_dim*4,
                                         dropout=0.1)
        self.transformer = nn.TransformerEncoder(enc, num_layers)
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(hidden_dim, hidden_dim//2, 2,2),
            nn.BatchNorm2d(hidden_dim//2), nn.ReLU(inplace=True),
            nn.ConvTranspose2d(hidden_dim//2, hidden_dim//4, 2,2),
            nn.BatchNorm2d(hidden_dim//4), nn.ReLU(inplace=True),
            nn.ConvTranspose2d(hidden_dim//4, 1, 2,2),
        )
    def forward(self, x, w_seq):
        feats   = self.vit.forward_features(x)       # [B,197,768]
        cls_tok = feats[:, :1]                       # [B,1,768]
        patch_t = feats[:, 1:]                       # [B,196,768]
        _, h_n  = self.weather_encoder(w_seq)        # [1,B,768]
        w_tok   = h_n.squeeze(0).unsqueeze(1)        # [B,1,768]
        tokens  = torch.cat([patch_t, w_tok, cls_tok],1)
        t       = self.transformer(tokens.permute(1,0,2)).permute(1,0,2)
        out_tok = t[:, :-2, :]                       # [B,196,768]
        B,N,D    = out_tok.shape
        G        = int(math.sqrt(N))
        x_small  = out_tok.transpose(1,2).view(B,D,G,G)
        return self.deconv(x_small)                  # [B,1,224,224]

In [ ]:
# Cell 3 ▶ Load checkpoint & Evaluate RMSE
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = PretrainedViTLSTModel(weather_dim=5).to(device)

# load the last checkpoint
ckpt = torch.load("/content/drive/MyDrive/Model_vit_gru_Checkpoints/vit_gru_epoch10.pt",
                  map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

mse = nn.MSELoss(reduction='sum')
total_loss = 0.0
total_pixels = 0

with torch.no_grad():
    for imgs, w_seq, tars in test_loader:
        imgs, w_seq, tars = imgs.to(device), w_seq.to(device), tars.to(device)
        preds = model(imgs, w_seq)
        total_loss   += mse(preds, tars).item()
        total_pixels += imgs.size(0) * preds.size(2) * preds.size(3)

rmse = math.sqrt(total_loss / total_pixels)
print(f"Test RMSE (per pixel): {rmse:.3f}°C")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Model_vit_gru_Checkpoints/vit_gru_epoch10.pt'